In [1]:
import pandas as pd
from gensim.models import Doc2Vec
import numpy as np

from string import digits
data=pd.read_excel("clean_article.xlsx")
data.dropna(how='any',inplace=True)
data.shape
data.head()

,Title,abstract,InterestValue,CiteCount,RecomCount,ReadCount,ResearcherURL,title_abs
0,Approximate Analytical Solutions of Space-Frac...,The main goal in this work is to establish a n...,13.9,0,5,275,https://www.researchgate.net/profile/Shirzoi_A...,approximate analytical solutions of space frac...
1,Poroelastic analysis on mesoscopic flow intera...,The wave-induced interlayer flow between mesos...,1.2,0,1,35,https://www.researchgate.net/profile/Chenghao_Cao,poroelastic analysis on mesoscopic flow intera...
2,A nonlinear method for multiparameter inversio...,Multiparameter inversion for pre-stack seismic...,2.6,4,0,22,https://www.researchgate.net/profile/Chenghao_Cao,a nonlinear method for multiparameter inversio...
3,Prestack density inversion using the Fatti equ...,Simultaneous prestack inversion is based on th...,3.0,2,0,59,https://www.researchgate.net/profile/Chenghao_Cao,prestack density inversion using the fatti equ...
4,Interaction of multiple courses of wave‐induce...,Different theoretical and laboratory studies o...,0.6,0,0,33,https://www.researchgate.net/profile/Chenghao_Cao,interaction of multiple courses of wave induce...


In [2]:
def getname(x):
    x=x.replace('https://www.researchgate.net/profile/','')
    x=x.replace('_',' ')
#     remove_digits = str.maketrans('', '', digits)
#     x=x.translate(remove_digits)
    return x
    

In [3]:
data['Researcher']=data['ResearcherURL'].apply(lambda x: getname(x))

In [4]:
newdata=data.groupby(data['Researcher'])['title_abs'].apply(lambda x:' '.join(x))

In [5]:
pd.DataFrame(newdata).head()

,title_abs
Researcher,
Abduelrahman Omer,cu n codoped tio2 prepared by the sol gel meth...
Abdul Wahab100,seismic behavior of renovated rc frame structu...
Abdulwasea Alkhazzan,existence and stability of a positive solution...
Abednego Acheampong,a comprehensive study of optimal linear pre co...
Ahmed Elshaikh,the potential role for hydro economic projects...


In [6]:
pd.DataFrame(newdata).to_excel('user_paper.xlsx')

In [7]:
model = Doc2Vec.load("Doc2Vec.model")

In [8]:
def to_review_vector(text):
    try:
        words = text.split(" ")
    except:
        words=[text]
    newword=[]
    for i in words:
        try:
            model[i]
            newword.append(i)
        except:
            pass
           
    atemp=np.asarray([model[w] for w in newword ],dtype='float32').mean(axis=0)
    if atemp.shape==():
        print(text)
    else:
         return atemp

In [9]:
data=pd.read_excel('user_paper.xlsx')
data.head()

,Researcher,title_abs
0,Abduelrahman Omer,cu n codoped tio2 prepared by the sol gel meth...
1,Abdul Wahab100,seismic behavior of renovated rc frame structu...
2,Abdulwasea Alkhazzan,existence and stability of a positive solution...
3,Abednego Acheampong,a comprehensive study of optimal linear pre co...
4,Ahmed Elshaikh,the potential role for hydro economic projects...


In [10]:
train_data_Doc2Vec = [to_review_vector(text) for text in data['title_abs']]

In [11]:
def cosine(_vec1, _vec2):
  
  return float(np.sum(_vec1*_vec2))/(np.linalg.norm(_vec1)*np.linalg.norm(_vec2))

In [12]:
Similarity_matrix=[]
for i in range(len(train_data_Doc2Vec)):
    if i%10==0:
        print(i)
    Similarity_matrix_axis=[]
    for j in range(len(train_data_Doc2Vec)):
        Similarity_matrix_axis.append(cosine(train_data_Doc2Vec[i], train_data_Doc2Vec[j]))
    Similarity_matrix.append(Similarity_matrix_axis)
    

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910


In [13]:
m=np.array(Similarity_matrix)
np.save('Similarity_matrix.npy',m)

In [14]:
a=np.load('Similarity_matrix.npy')
Similarity_matrix=a.tolist()